In [1]:
import torch
import torchvision
from torchvision import models
from torchvision import transforms

import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

import os
import time
import pathlib
import sys

In [2]:
repo_path = pathlib.Path(os.getcwd())
base_dir = repo_path / "Neural Style Transfer/"
base_dir

PosixPath('/Users/vineetmahajan/Code/AI/Projects/Hello/Hello-CNN/Neural Style Transfer')

In [3]:
os.chdir(base_dir)

In [4]:
content_dir_path = base_dir / "content images/"
style_dir_path = base_dir / "style images/"
content_images_path = [
    content_dir_path / img 
    for img in os.listdir(content_dir_path) 
    if img.split(".")[-1] 
    in ["jpg", "jpeg", "png"]
]
style_images_path = [
    style_dir_path / img
    for img in os.listdir(style_dir_path)
    if img.split(".")[-1] 
    in ["jpg", "jpeg", "png"]
]

content_images_path, style_images_path

([PosixPath('/Users/vineetmahajan/Code/AI/Projects/Hello/Hello-CNN/Neural Style Transfer/content images/The-Boy.jpeg'),
  PosixPath('/Users/vineetmahajan/Code/AI/Projects/Hello/Hello-CNN/Neural Style Transfer/content images/The-Guardian-2.jpg'),
  PosixPath('/Users/vineetmahajan/Code/AI/Projects/Hello/Hello-CNN/Neural Style Transfer/content images/The-Guardian.jpg')],
 [PosixPath('/Users/vineetmahajan/Code/AI/Projects/Hello/Hello-CNN/Neural Style Transfer/style images/red-abstract.jpg'),
  PosixPath('/Users/vineetmahajan/Code/AI/Projects/Hello/Hello-CNN/Neural Style Transfer/style images/puzzled_women.jpg'),
  PosixPath('/Users/vineetmahajan/Code/AI/Projects/Hello/Hello-CNN/Neural Style Transfer/style images/emotional-joshua-miels.jpg'),
  PosixPath('/Users/vineetmahajan/Code/AI/Projects/Hello/Hello-CNN/Neural Style Transfer/style images/van-gogh.jpg')])

In [5]:

device = torch.device('mps')

def load_image(
    image_path: str,
    transform=None,
    max_size=None,
    shape=None,
    pad=False
):
    """Load an image and convert it to a torch tensor."""
    image = Image.open(image_path)

    if max_size:
        scale = max_size / max(image.size)
        size = np.array(image.size) * scale
        image = image.resize(size.astype(int), Image.Resampling.LANCZOS)
        
    if transform:
        image = transform(image)#.unsqueeze(0)

    if shape:
        image = image.resize(
            shape,
            Image.LANCZOS
        )
    if pad:
        _, h, w = image.size()
        max_hw = max(h, w)
        ph = (max_hw - h) / 2
        pw = (max_hw - w) / 2
        lwp = int(pw if pw % 1 == 0 else pw + 0.5)
        rwp = int(pw // 1)
        thp = int(ph if ph % 1 == 0 else ph + 0.5)
        bhp = int(ph // 1)
        image = transforms.functional.pad(
            image,
            padding=(lwp, thp, rwp, bhp),
            fill=0,
            padding_mode="constant"
        )
        

    return image.numpy()

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406),
                         std=(0.229, 0.224, 0.225))])

# content = load_image( content_images[1], transform, max_size=400, pad=True)
# style = load_image(config.style, transform, shape=[content.size(2), content.size(3)])
content_images = torch.Tensor([
    load_image(
        path_,
        transform,
        max_size=400,
        pad=True
    ) for path_ in content_images_path
])

style_image = load_image(
        style_images_path[-1],
        transform,
        max_size=400,
        pad=True
)

generated_images = content_images.clone()

content_images = content_images.to(device)
style_image = torch.Tensor(style_image).to(device)
generated_images = generated_images.to(device).requires_grad_(True)


/var/folders/10/d1l6p8_s53q1w912xq9jh2_80000gn/T/ipykernel_56541/1681988121.py:52: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /Users/runner/miniforge3/conda-bld/pytorch-recipe_1664817727684/work/torch/csrc/utils/tensor_new.cpp:204.)
  content_images = torch.Tensor([


In [7]:
class VGGNet(nn.Module):
    def __init__(self):
        """Select conv1_1 ~ conv5_1 activation maps."""
        super(VGGNet, self).__init__()
        self.select = ['0', '5', '10', '19', '28']
        self.vgg = models.vgg19(pretrained=True).features

    def forward(self, x):
        """Extract multiple convolutional feature maps."""
        features = []
        for name, layer in self.vgg._modules.items():
            x = layer(x)
            if name in self.select:
                features.append(x)
        return features

num_images = generated_images.size(0)
generated_dir_path = base_dir / "generated images/"
generated_images_path = [
    generated_dir_path / (c_path.name.split(".")[0] +"-"+ style_images_path[-1].name.split(".")[0])
    for c_path in content_images_path
]
[
    os.makedirs(path_, exist_ok=True)
    for path_ in generated_images_path
    
]

[None, None, None]

In [18]:
x = torch.Tensor(1)
x.to(torch.float64)

tensor([0.], dtype=torch.float64)

In [19]:
optimizer = torch.optim.Adam([generated_images], lr=0.003, betas=[0.5, 0.999])
backbone = VGGNet().to(device).eval()
config = {
    "content_weight": 1,
    "style_weight": 100,
    "log_step": 10,
    "total_step": 1000,
    "save_step": 100,
}
# for step in range(config["total_step"]):
for step in range(1):
    generated_features = backbone(generated_images)
    content_features = backbone(content_images)
    style_features = backbone(style_image)
    
    content_loss = style_loss = torch.zeros(num_images, dtype=torch.float64).to(device)
    for f_gen, f_con, f_stl in zip(generated_features, content_features, style_features):
        content_loss += torch.mean((f_gen - f_con)**2, axis=[1, 2, 3]).to(torch.float64)
        
        n, c, h, w = f_gen.size()
        f_gen = f_gen.view(n, c, h*w)
        f_stl = f_stl.view(c, h*w)
        
        f_gen = torch.bmm(f_gen, f_gen.transpose(1, 2)).to(torch.float64)
        f_stl = torch.mm(f_stl, f_stl.t()).to(torch.float64)
        
        style_loss += torch.mean((f_gen - f_stl)**2, axis=[1, 2])
        
        
    loss = config["content_weight"] * content_loss + config["style_weight"] * style_loss
    loss = loss.mean()
    
    if (step+1) % config["log_step"] == 0:
        print(f"Step [{step+1}/{config['total_step']}]\n Loss: {loss.item():.4f}\n")
    if (step+1) % config["save_step"] == 0:
        
        generated_raw_images = generated_images.clone().detach().cpu()
        denorm = transforms.Normalize((-2.12, -2.04, -1.80), (4.37, 4.46, 4.44))
        generated_raw_images = denorm(generated_raw_images).clamp_(0, 1)
        [
            torchvision.utils.save_image(
                img, 
                path_ / f"output-{step+1}.png"
            )
            for img, path_
            in zip(generated_raw_images, generated_images_path)
        ]
        

/Users/vineetmahajan/.pyenv/versions/miniconda3-4.12/envs/torch_gpu/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/Users/vineetmahajan/.pyenv/versions/miniconda3-4.12/envs/torch_gpu/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


TypeError: Cannot convert a MPS Tensor to float64 dtype as the MPS framework doesn't support float64. Please use float32 instead.

In [10]:
len(generated_raw_images)

NameError: name 'generated_raw_images' is not defined

In [21]:
generated_images_path

[PosixPath('/Users/vineetmahajan/Code/AI/Projects/Hello/Hello-CNN/Neural Style Transfer/generated images/The-Boy-van-gogh'),
 PosixPath('/Users/vineetmahajan/Code/AI/Projects/Hello/Hello-CNN/Neural Style Transfer/generated images/The-Guardian-2-van-gogh'),
 PosixPath('/Users/vineetmahajan/Code/AI/Projects/Hello/Hello-CNN/Neural Style Transfer/generated images/The-Guardian-van-gogh')]

In [22]:
generated_raw_images = generated_images.clone().detach().cpu()
denorm = transforms.Normalize((-2.12, -2.04, -1.80), (4.37, 4.46, 4.44))
generated_raw_images = denorm(generated_raw_images).clamp_(0, 1)
generated_raw_images = [
    torchvision.utils.save_image(
        img, 
        path_ / f"output-{step+1}.png"
    )
    for img, path_
    in zip(generated_raw_images, generated_images_path)
]

generated_raw_images

FileNotFoundError: [Errno 2] No such file or directory: '/Users/vineetmahajan/Code/AI/Projects/Hello/Hello-CNN/Neural Style Transfer/generated images/The-Boy-van-gogh/output-1.png'

In [90]:
optimizer.zero_grad()
loss.backward()
optimizer.step()

In [45]:
content_loss += torch.mean((f_gen - f_con)**2, axis=[1, 2, 3])
n, c, h, w = f_gen.size()
x_gen = f_gen.view(n, c, -1)
y_gen = f_stl.view(c, -1)
# f_gen.shape
x_gen.shape

torch.Size([3, 512, 625])

In [46]:
x = torch.randn(3, 512, 625)

# Get the number of matrices in the tensor
num_matrices = x.shape[0]

# Initialize an empty tensor to store the result
result = torch.empty(num_matrices, x.shape[1], x.shape[1])

# Perform matrix multiplication for each matrix in the tensor
for i in range(num_matrices):
    matrix = x[i]  # Get the matrix at index i
    result[i] = torch.matmul(matrix, matrix.t())  # Perform matrix multiplication with its transpose

# Print the resulting tensor of shape [3, 512, 512]
print(result.shape)

torch.Size([3, 512, 512])


In [66]:
x = torch.randn(3, 512, 625)
x_transposed = x.transpose(1, 2)
result = torch.bmm(x, x_transposed)
print(result.shape)

torch.Size([3, 512, 512])


In [55]:
x_transposed.shape

torch.Size([3, 625, 512])

In [59]:
t = torch.randint(0, 10, (2, 3, 4))
t

tensor([[[3, 1, 8, 8],
         [7, 2, 0, 0],
         [2, 2, 0, 9]],

        [[7, 6, 2, 9],
         [1, 8, 9, 1],
         [3, 9, 9, 2]]])

tensor([[[3, 7, 2],
         [1, 2, 2],
         [8, 0, 0],
         [8, 0, 9]],

        [[7, 1, 3],
         [6, 8, 9],
         [2, 9, 9],
         [9, 1, 2]]])